In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

df = pd.read_csv('/content/drive/MyDrive/Copyoffer2013.csv')
print(df.head())

X_train = []
y_train = []
X_test = []
y_test = []

for index, row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

X_train = np.array(X_train, dtype='uint8')
y_train = np.array(y_train, dtype='uint8')
X_test = np.array(X_test, dtype='uint8')
y_test = np.array(y_test, dtype='uint8')

# feature selection
selector = SelectKBest(f_classif, k=200)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# convert labels to categorical variables
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# vary the number of neighbors and check the accuracy
accuracy_list = []
for i in range(1, 21):
    # create the KNN model
    knn = KNeighborsClassifier(n_neighbors=i, weights='distance')

    # fit the model to the training data
    knn.fit(X_train, y_train)

    # predict labels for test set
    y_pred_labels = knn.predict(X_test)

    # compute and print the accuracy
    accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_labels, axis=1))
    accuracy_list.append(accuracy)


# compute and print the accuracy using the score method of KNeighborsClassifier
accuracy = knn.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))